## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data_customers = pd.read_csv('/datasets/data.csv')
data_customers.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

Изучив таблицу с данными я обнаружил множество проблем, среди которых: 
1) В столбце children могут быть отрицательные значения.
2) В столбце days_employed также могут быть отрицательные значения.
3) В столбце education поля указаны в разных регистрах.
4) В столбце days_employed есть пропущенные данные.
5) В столбце total_income есть пропущенные данные.

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим какие столбцы содержат пропущенные значения. Столбцы days_employed и total_income могут содержать пропущенные данные по причине того, что люди никогда не работали.

In [2]:
columns_with_holes = data_customers.columns[data_customers.isna().any()].tolist()
columns_with_holes

['days_employed', 'total_income']

Выясним сколько и какой процент составляют пропущенные данные от общего количества.

In [3]:
for column in columns_with_holes:
    number_of_holes = data_customers[column].isnull().sum()
    total_number = data_customers[column].count()
    print(column, number_of_holes, '{:.2f}%'.format(number_of_holes / total_number * 100))

days_employed 2174 11.23%
total_income 2174 11.23%


Так как пропущенных данных достаточно много, мы не можем их просто удалить.
Функция fill_total_income для строк с пропущенными данными в столбце total_income возвращает медианное значение этого столбца для данной должности.

In [4]:
median_total_income = data_customers.groupby('income_type')['total_income'].median()

def fill_total_income(row):
    income_type = row['income_type']
    total_income = row['total_income']
    if pd.isnull(total_income):
        return median_total_income[income_type]
    return total_income

Заполняем пропуски в столбце total_income. Столбец days_employed оставляем нетронутым, так как мы не собираемся никак его использовать в аналитике.

In [5]:
data_customers['total_income'] = data_customers.apply(fill_total_income, axis = 1)

Проверяем пропуски в столбце total_income и убеждаемся, что их ровно 0.

In [6]:
data_customers['total_income'].isna().sum()

0

### Вывод

Данные могут быть не идеальными и мы должны их правильно предобрабатывать, чтобы получить как можно лучший результат при анализе.

### Замена типа данных

Выводим информацию о таблице и узнаем какие столбцы представлены вещественным типом.

In [7]:
data_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Так как мы не собираемся работать со столбцом days_employed, то к целочисленному типу приводим только столбец total_income. Также нам нужно убрать отрицательные значение из столбца дети. Заменим их на 0, так как отрицательные значения равносильны отсутсвию детей. Возможные причины появления таких значений: смерть ребенка, ошибка при составлении таблицы.

In [8]:
data_customers['total_income'] = data_customers['total_income'].astype(int)
data_customers['children'] = data_customers['children'].apply(lambda children: children if children >= 0 else 0) 

Выводим информацию о таблице, что бы убедиться, что тип данных у столбца total_income теперь int64.

In [9]:
data_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Вывод

Данные в таблицах могут не всегда соответствовать нашим ожиданиям и перед работай над анализом всегда нужно сперва убедиться, все ли столбцы соответсвуют ожидаемым типам данных.

### Обработка дубликатов

Проверяем таблицу на дубликаты.

In [10]:
data_customers.duplicated().sum()

54

Удаляем все лишние дубликаты и делаем reset_index, что бы обновить индексы.

In [11]:
data_customers = data_customers.drop_duplicates().reset_index(drop=True)

Убеждаемся в том, что дубликатов больше нет

In [12]:
data_customers.duplicated().sum()

0

### Вывод

Таблицы могут содержать повторяющиеся данные, которые необходимо удалить для проведения правильно анализа.

### Лемматизация

Выводим все уникальные элементы столбца purpose, чтобы выделить возможные 'леммы'.

In [13]:
data_customers['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Множество common_purposes содержит набор возможных лемм, функция get_common_purpose ставит в соответствие элементу столбца purpose соответствующую лемму.

In [14]:
from pymystem3 import Mystem
m = Mystem()

common_purposes = set(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'])
def get_common_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        if lemma in common_purposes:
            return lemma
    return purpose

Создаем дополнительный столбец common_purpose, в который записываем лемму на основании purpose.

In [15]:
data_customers['common_purpose'] = data_customers['purpose'].apply(get_common_purpose)

Выводим все уникальные значения столбца common_purpose, чтобы убедиться, что все леммы соответствуют элементам набора common_purposes.

In [16]:
data_customers['common_purpose'].unique()

array(['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'],
      dtype=object)

### Вывод

Лемматизация позволяет нам выделять более общие 'группы', что может быть очень удобным при анализе.

### Категоризация данных

Создаем функцию get_pivot_relation для получения сводной таблицы на основании category и debt.

In [17]:
def get_pivot_relation(df, category):
    return df.pivot_table(index=category, values='debt')

In [18]:
children_relation = get_pivot_relation(data_customers, 'children')
children_relation

,debt
children,
0,0.075173
1,0.092327
2,0.094542
3,0.081818
4,0.097561
5,0.000000
20,0.105263


Выясняем связь между семейным статусом и долгом.

In [36]:
family_relation = get_pivot_relation(data_customers, 'family_status')
family_relation

,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093202
женат / замужем,0.075421


Выясняем связь между доходом и долгом.

In [37]:
total_income_relation = get_pivot_relation(data_customers, 'total_income')
total_income_relation

,debt
total_income,
20667,1.0
21205,0.0
21367,0.0
21695,0.0
21895,0.0
...,...
1711309,0.0
1715018,0.0
1726276,0.0


Выясняем связь между целью и долгом.

In [39]:
purpose_relation = get_pivot_relation(data_customers, 'common_purpose')
purpose_relation

,debt
common_purpose,
автомобиль,0.093547
жилье,0.069043
недвижимость,0.074610
образование,0.092177
свадьба,0.079657


### Вывод

Категоризация данных позволяет определить общие свойства объектов и на их основе сделать аналитические выводы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Из сводной таблицы children_relation мы можем видеть, что сначала при увеличение количества
детей до 2 долг увеличивается, но затем при 3 детях он падает, при 4 поднимается и при 5
падает до нуля. Так как мы не видим четкого возрастания или убывания долга при увеличении
количества детей, то мы не можем сделать однозначные выводы: влияет ли количество детей 
клиента на факт погашения кредита в срок.

### Вывод

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Из сводной таблицы family_relation мы можем видеть, что люди с семейным статусом 
вдовец или вдова имеют наибольшую вероятность возврата кредита в срок, затем идут женатые и
в разводе люди, затем люди в гражданском браке и не женатые. Таким образом можно сказать,
что определенная зависимость все таки присутствует.

### Вывод

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Из сводной таблицы total_income_relation мы можем видеть, что люди с наименьшим уровнем
дохода(20667) и с большим уровнем дохода(2200852) могут представлять потенциальные риски
для банка. Однако исходя из этих данных невозможно сделать однозначные выводы, так как люди с доходами в окрестности данных точек не имеют долгов.

### Вывод

- Как разные цели кредита влияют на его возврат в срок?

Из сводной таблицы purpose_relation мы можем видеть, что люди берущие кредит на жилье имеют
наибольшую вероятность возврата кредита в срок, затем берущие на недвижимость и свадьбу, затем на автомобиль и образование.

### Шаг 4. Общий вывод

Аналитика данных включает множество различных связанных между собой процессов, таких как:
обработка пропусков, замена типов данных, удаление дубликатов, лемматизация, категоризация данных. Каждый из таких процессов одновременно независим и зависим от других, так как если что-то из этого не сделать, то неправильные данные будут накапливаться как снежный ком от стадии к стадии и в конце концов мы сделаем неправильные выводы. На примере данного проекта
я прочувствовал насколько важен каждый из этих этапов и научился делать соответствующие 
выводы из данных.